## ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Convolutional Neural Networks I

### LEARNING OBJECTIVES
_By the end of this lesson, students should be able to:_
- Build convolutional neural networks in Keras.

We'll recreate a very similar neural network to the example provided at the end of the notes.

In [1]:
# 1. Import libraries and modules
import numpy as np
np.random.seed(123)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
 
# 2. Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

/Users/mattbrems/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/mattbrems/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
X_train[0:5]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [4]:
X_train[0][5]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
        18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
         0,   0], dtype=uint8)

In [5]:
# 3. Preprocess our input data.

# Reshaping here will make the array work more nicely with Keras.
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Switching from int8 to float32.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Convert 0-255 range to 0-1 range.
X_train /= 255
X_test /= 255

In [6]:
X_train[0][5]

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01176471],
       [0.07058824],
       [0.07058824],
       [0.07058824],
       [0.49411765],
       [0.53333336],
       [0.6862745 ],
       [0.10196079],
       [0.6509804 ],
       [1.        ],
       [0.96862745],
       [0.49803922],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]], dtype=float32)

In [7]:
# 4. Preprocess our class labels.

y_train[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [8]:
# Convert individual numbers in y_train to categorical output.

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [9]:
Y_train[0:10]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

60 parameters (in Conv2D layer)

each filter was 3x3. <-- 9 parameters
6 filters <-- 6 filters x 9 parameters per filter = 54 parameters
for each channel (a.k.a. result from filter), we have 1 bias parameter <-- 1 parameter per filter x 6 filters = 6 parameters

Answer: 54 parameters (within filters) + 6 parameters (bias) = 60 parameters

16 filters
3x3 kernel size

16 filters x 9 parameters per filter = 144 filters
144 filters per channel x 6 channels = 864 filters
16 filters in the "new layer," we have 16 bias parameters
--> 864 + 16 = 880

In [10]:
## 5. Define model architecture.

model = Sequential() # Instantiate our NN in the same way.

model.add(Convolution2D(filters = 6,     # number of filters/channels
                        kernel_size = 3, # filters are 3x3
                        activation = 'relu',
                        input_shape = (28, 28, 1))) # (height, width, depth)

model.add(MaxPooling2D(pool_size = (2,2))) # 2x2 filter
# by default, MaxPooling2D will pool over non-overlapping regions

model.add(Convolution2D(filters = 16,
                        kernel_size = 3,
                        activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten()) # converts "box" to vertical array of nodes

model.add(Dense(128, activation = 'relu')) # finally at the FC layer(s)

model.add(Dropout(0.5)) # randomly dropping out 50% of 
                        # nodes during training

model.add(Dense(10, activation = 'softmax'))

# 10 nodes because 10 possible outputs/values for Y {0, 1, ..., 9}
# softmax activation because it ensures my predictions are
# non-negative and they sum to 100% for a given observation

In [11]:
# 6. Compile model.

model.compile(loss = 'categorical_crossentropy',
              # common choice for unordered discrete predictions
              optimizer = 'adam',
              # more sophisticated version of gradient descent
              metrics = ['accuracy'])

In [12]:
# 7. Fit model on the training data.

model.fit(X_train,
          Y_train,
          batch_size = 32, 
          epochs = 10,
          verbose = 1)

Epoch 1/10
60000/60000 [==============================] - 42s 702us/step - loss: 0.2935 - acc: 0.9101
Epoch 2/10
60000/60000 [==============================] - 30s 498us/step - loss: 0.1071 - acc: 0.9678
Epoch 3/10
60000/60000 [==============================] - 28s 466us/step - loss: 0.0831 - acc: 0.9752
Epoch 4/10
60000/60000 [==============================] - 26s 434us/step - loss: 0.0705 - acc: 0.9782
Epoch 5/10
60000/60000 [==============================] - 29s 483us/step - loss: 0.0625 - acc: 0.9806
Epoch 6/10
60000/60000 [==============================] - 27s 451us/step - loss: 0.0548 - acc: 0.9832
Epoch 7/10
60000/60000 [==============================] - 28s 461us/step - loss: 0.0488 - acc: 0.9852
Epoch 8/10
60000/60000 [==============================] - 30s 500us/step - loss: 0.0468 - acc: 0.9856
Epoch 9/10
60000/60000 [==============================] - 31s 509us/step - loss: 0.0425 - acc: 0.9869
Epoch 10/10
60000/60000 [==============================] - 28s 472us/step - loss: 

In [13]:
# 8. Evaluate model on test data.

score = model.evaluate(X_test, Y_test, verbose = 1)
labels = model.metrics_names

10000/10000 [==============================] - 3s 328us/step


In [14]:
print(str(labels[0]) + ": " + str(score[0]))
print(str(labels[1]) + ": " + str(score[1]))

loss: 0.03254934827641973
acc: 0.9891


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               51328     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

## Conclusion

<details><summary>Why are neural networks better equipped to handle image data than non-neural networks?
</summary>
```
Neural networks are naturally set up to consider interactions among features.
```
</details>

<details><summary>Why are **convolutional neural networks** better equipped to handle image data than non-CNNs?
</summary>
```
CNNs are naturally set up to consider interactions among "close pixels" only and drastically cuts down the number of parameters needed to learn through parameter sharing.
```
</details>

